In [31]:
from os import listdir
import matplotlib.pyplot as plt
import pandas as pd
import re
from dtw import dtw
from scipy.spatial import distance
import pyopenms
from operator import itemgetter
%matplotlib inline

In [21]:
#Combines several spectra (separated by time) into a single overall spectrum for that experiment
def sum_spectra(spectra):
    summed_peaks = {}
    for spectrum in spectra:
        mz, intensity = spectrum.get_peaks()
        for i in range(len(mz)):
            current_mz = mz[i]
            current_intensity = intensity[i]
            #This accounts for duplicate peak entries with the same m/z value (and adds their intensities together)
            if current_mz in summed_peaks:
                summed_peaks[current_mz] += current_intensity
            else:
                summed_peaks[current_mz] = current_intensity
    return(summed_peaks)

#Returns a dictionary of peaks obtained by subtracting the peaks of the unbound spectrum (Ub) away from the peaks of the bound spectrum (Ub + either C, O, or T)
def get_difference(bound, unbound):
    bound_peaks = {}
    unbound_peaks = {}
    difference_peaks = {}
    for mz, intensity in bound.items():
        if mz in bound_peaks:
            bound_peaks[mz] += intensity
        else:
            bound_peaks[mz] = intensity
    for mz, intensity in unbound.items():
        if mz in unbound_peaks:
            unbound_peaks[mz] += intensity
        else:
            unbound_peaks[mz] = intensity
    #The values in both spectra are normalized by dividing by the maximum value, so the maximum intensity is set to 1
    bound_max = max(bound_peaks.values())
    unbound_max = max(unbound_peaks.values())
    for mz, i in bound_peaks.items():
        difference_peaks[mz] = i / bound_max
    for mz, i in unbound_peaks.items():
        if mz in difference_peaks:
            difference_peaks[mz] -= i / unbound_max
        else:
            difference_peaks[mz] = -i / unbound_max
    return(difference_peaks)

#Finds the closest peak (in the theoretical spectrum) for a given peak in the difference spectrum
def find_closest(peak):
    #If the m/z of the given peak is lower than any in the theoretical spectrum, return the peak with the lowest m/z in the theoretical spectrum
    if peak[0] < theoretical[0][0]:
        return(theoretical[0])
    #If the m/z of the given peak is higher than any in the theoretical spectrum, return the peak with the highest m/z in the theoretical spectrum
    elif peak[0] > theoretical[-1][0]:
        return(theoretical[-1])
    else:
        left = 0
        right = 0
        index = 0
        #Find the closest peak on either side of the given peak
        while left == 0 :
            current = theoretical[index]
            if current[0] == peak[0]:
                return(current)
            elif current[0] > peak[0]:
                right = current
                left = theoretical[index - 1]
            index += 1
        #Return the peak on the side that is closer to the one given
        left_closer = peak[0] - left[0] < right[0] - peak[0]
        if left_closer:
            return(left)
        else:
            return(right)

## Main

In [8]:
#extract files
input_data_url = 'Data//CSV Data//'
input_files = sorted(listdir(input_data_url))
input_files = [file for file in input_files if file not in ['.DS_Store', 'demo_4.csv']]

#unbound/bound files
unbound_pattern = re.compile("u")
unbound_file_matches = [file for file in input_files if unbound_pattern.match(file)] 
bound_file_matches = [file for file in input_files if file not in unbound_file_matches]

In [9]:
#Open the file with the spectra and read the contents
unbound_df = pd.read_csv("Data//CSV Data//ub_1.csv")
bound_df = pd.read_csv("Data//CSV Data//c_1.csv")

In [23]:
#EXPERIMENTAL DATA

#extract bound and unbound m/z and intensity
bound_mz = list(bound_df['m/z'].values)
bound_intensity = list(bound_df['intensity'].values)
unbound_mz = list(unbound_df['m/z'].values)
unbound_intensity = list(unbound_df['intensity'].values)

#Create experiments
bound_exp = pyopenms.MSExperiment()
bound_spectrum = pyopenms.MSSpectrum()
unbound_exp = pyopenms.MSExperiment()
unbound_spectrum = pyopenms.MSSpectrum()

#Update the experiment with the bound or unbound data, then store it in a file
unbound_spectrum.set_peaks([unbound_mz, unbound_intensity])
unbound_exp.setSpectra([unbound_spectrum])
bound_spectrum.set_peaks([bound_mz, bound_intensity])
bound_exp.setSpectra([bound_spectrum])

#combine spectrum
bound_spectrum = sum_spectra(bound_exp.getSpectra())
unbound_spectrum = sum_spectra(unbound_exp.getSpectra())

#In theory, subtracting the unbound spectrum from the bound spectrum should return the effects of the binding with the platin
binding_effect = get_difference(bound_spectrum, unbound_spectrum)

In [25]:
#THEORETICAL DATA

#Convert the string representation of Ubiquitin into an amino acid sequence object
ubiquitin = pyopenms.AASequence.fromString("MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG")
tsg = pyopenms.TheoreticalSpectrumGenerator()
spectrum = pyopenms.MSSpectrum()
#Initialise parameters object and set the values for parameters
#To change which parameters are set to true (false is default), specific lines can be commented out and vice versa
parameters = pyopenms.Param()
parameters.setValue(b"add_isotopes", b"true", "")
#parameters.setValue(b"add_losses", b"true", "")
parameters.setValue(b"add_b_ions", b"true", "")
parameters.setValue(b"add_y_ions", b"true", "")
parameters.setValue(b"add_a_ions", b"true", "")
parameters.setValue(b"add_c_ions", b"true", "")
parameters.setValue(b"add_x_ions", b"true", "")
parameters.setValue(b"add_z_ions", b"true", "")
#parameters.setValue(b"add_precursor_peaks", b"true", "")
#parameters.setValue(b"add_all_precursor_charges", b"true", "")
#parameters.setValue(b"add_first_prefix_ion", b"true", "")
parameters.setValue(b"add_metainfo", b"true", "")
tsg.setParameters(parameters)
#Generate the theoretical spectrum of Ubiquitin
tsg.getSpectrum(spectrum, ubiquitin, 1, 2)

In [27]:
#The theoretical spectrum is converted from a pair of lists to a single list of [m/z, i] items
theoretical = []
for i in range(len(spectrum.get_peaks()[0])):
    current_mz = spectrum.get_peaks()[0][i]
    current_intensity = spectrum.get_peaks()[1][i]
    theoretical.append([current_mz, current_intensity])

In [32]:
#Stores the peaks of the difference spectrum in a list of [m/z, i] items
difference = []
for mz, i in binding_effect.items():
    if i < 0:
        difference.append([mz,-i])
        
#Sort the difference spectrum by intensity values from largest to smallest
difference = sorted(difference, key=itemgetter(1), reverse=True)

In [30]:
#DTW bound and unbound 
euclidean_distance = lambda mass_spec_1, mass_spec_2: np.abs(mass_spec_1 - mass_spec_2)
d, cost_matrix, acc_cost_matrix, path = dtw(x, y, dist=euclidean_distance)

In [23]:
warpedScreening1 = referenceScreening[path[0]]
warpedScreening2 = referenceScreening[path[1]]

0        0.005005
1        0.010117
2        0.009681
3        0.009465
4        0.009649
           ...   
48120    0.000419
48121    0.000539
48122    0.000721
48123    0.000937
48124    0.001155
Name: intensity, Length: 48125, dtype: float64

0         0.003937
1         0.001773
2         0.001385
3         0.000889
4         0.000796
            ...   
752564    0.000913
752565    0.001003
752566    0.000956
752567    0.000779
752568    0.000541
Name: intensity, Length: 752569, dtype: float64

### Helper Functions

In [12]:
def mass_charge_concatenation(data):
    
    #Initialise a dictionary to store the peaks
    peaks_dict = {}
    #Process each line of the text file and store the data in the dictionary
    for idx in range(0, data.shape[0]):
        current_mz = data.loc[idx, 'm/z']
        current_intensity = data.loc[idx, 'intensity']
        #Multiple peaks with the same m/z are present (usually separated by time) then sum them and if not 
        #create new peak
        if current_mz in peaks_dict.keys():
            peaks_dict[current_mz] += current_intensity
        else:
            peaks_dict[current_mz] = current_intensity
            
    return peaks_dict